In [1]:
# way to define spark context
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as SparkTypes

In [2]:
""" import os

# Escape backslashes, use raw string, wrap in quotes
os.environ["PYSPARK_SUBMIT_ARGS"] = r'--jars "E:/Projects/Mini Practice Project Data Pipeline with ETL, Validation, and Airflow/Spark_process/postgresql-42.7.5.jar" pyspark-shell'
 """

' import os\n\n# Escape backslashes, use raw string, wrap in quotes\nos.environ["PYSPARK_SUBMIT_ARGS"] = r\'--jars "E:/Projects/Mini Practice Project Data Pipeline with ETL, Validation, and Airflow/Spark_process/postgresql-42.7.5.jar" pyspark-shell\'\n '

In [3]:
spark = SparkSession.builder \
    .appName("CoronaProj") \
    .config("spark.jars", r"E:/Projects/Mini Practice Project Data Pipeline with ETL, Validation, and Airflow/Spark_process/postgresql-42.7.5.jar") \
    .getOrCreate()
    
sc = spark.sparkContext

In [4]:
spark.conf.set("spark.sql.session.timeZone", "Africa/Cairo")

In [5]:

df = spark.read.option("multiline", "true").json("data.json")

df.printSchema()

root
 |-- active: long (nullable = true)
 |-- activePerOneMillion: double (nullable = true)
 |-- cases: long (nullable = true)
 |-- casesPerOneMillion: long (nullable = true)
 |-- continent: string (nullable = true)
 |-- country: string (nullable = true)
 |-- countryInfo: struct (nullable = true)
 |    |-- _id: long (nullable = true)
 |    |-- flag: string (nullable = true)
 |    |-- iso2: string (nullable = true)
 |    |-- iso3: string (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- long: double (nullable = true)
 |-- critical: long (nullable = true)
 |-- criticalPerOneMillion: double (nullable = true)
 |-- deaths: long (nullable = true)
 |-- deathsPerOneMillion: long (nullable = true)
 |-- oneCasePerPeople: long (nullable = true)
 |-- oneDeathPerPeople: long (nullable = true)
 |-- oneTestPerPeople: long (nullable = true)
 |-- population: long (nullable = true)
 |-- recovered: long (nullable = true)
 |-- recoveredPerOneMillion: double (nullable = true)
 |-- tests: 

In [6]:
df.show(n=2,truncate=False)

+------+-------------------+------+------------------+---------+-----------+--------------------------------------------------------------------+--------+---------------------+------+-------------------+----------------+-----------------+----------------+----------+---------+----------------------+-------+------------------+----------+-----------+--------------+-------------+
|active|activePerOneMillion|cases |casesPerOneMillion|continent|country    |countryInfo                                                         |critical|criticalPerOneMillion|deaths|deathsPerOneMillion|oneCasePerPeople|oneDeathPerPeople|oneTestPerPeople|population|recovered|recoveredPerOneMillion|tests  |testsPerOneMillion|todayCases|todayDeaths|todayRecovered|updated      |
+------+-------------------+------+------------------+---------+-----------+--------------------------------------------------------------------+--------+---------------------+------+-------------------+----------------+-----------------+----

In [7]:
df2 = df.select(
    F.col("continent"),
    F.col("country"),
    F.col("cases"),
    F.col("deaths"),
    F.col("recovered"),
    F.from_unixtime(F.col("updated")/1000).cast("timestamp").alias("egypt_time"),
)

In [8]:
df2.show(n=5)

+---------+-----------+------+------+---------+-------------------+
|continent|    country| cases|deaths|recovered|         egypt_time|
+---------+-----------+------+------+---------+-------------------+
|     Asia|Afghanistan|234174|  7996|   211080|2025-04-19 18:43:55|
|   Europe|    Albania|334863|  3605|   330233|2025-04-19 18:43:55|
|   Africa|    Algeria|272010|  6881|   183061|2025-04-19 18:43:55|
|   Europe|    Andorra| 48015|   165|        0|2025-04-19 18:43:55|
|   Africa|     Angola|107327|  1937|   103419|2025-04-19 18:43:55|
+---------+-----------+------+------+---------+-------------------+
only showing top 5 rows



In [9]:
df_clean = df2.filter((F.col("cases") >= 0) & (F.col("deaths") >= 0) & (F.col("recovered") >= 0)).dropna()

In [10]:
total_countries = df_clean.select("country").count()
unique_countries = df_clean.select("country").agg(F.countDistinct("country")).collect()[0][0]

if total_countries == unique_countries:
    print("countries are unqiue")
else:
    print("Countries are not unique")

countries are unqiue


In [11]:
df_clean.show(n=4)

+---------+-----------+------+------+---------+-------------------+
|continent|    country| cases|deaths|recovered|         egypt_time|
+---------+-----------+------+------+---------+-------------------+
|     Asia|Afghanistan|234174|  7996|   211080|2025-04-19 18:43:55|
|   Europe|    Albania|334863|  3605|   330233|2025-04-19 18:43:55|
|   Africa|    Algeria|272010|  6881|   183061|2025-04-19 18:43:55|
|   Europe|    Andorra| 48015|   165|        0|2025-04-19 18:43:55|
+---------+-----------+------+------+---------+-------------------+
only showing top 4 rows



In [12]:
print(spark.sparkContext._jsc.sc().listJars())


Vector()


In [13]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="123456789"
)

cursor = conn.cursor()

# Check if the table exists
cursor.execute("""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE  table_schema = 'public' 
        AND    table_name   = 'corona_proj'
    );
""")

exists = cursor.fetchone()[0]
print(exists)
# Create the table if it doesn't exist
if not exists:
    cursor.execute("""
        CREATE TABLE public.corona_proj (
            continent CHAR(15),
            country CHAR(15),
            cases INT,
            deaths INT,
            recovered INT,
            egypt_time TIMESTAMP
        );
    """)
    conn.commit()

cursor.close()
conn.close()


True


In [14]:
df_clean.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://localhost:5432/postgres") \
  .option("dbtable", "public.Corona_Proj") \
  .option("user", "postgres") \
  .option("password", "123456789") \
  .option("driver", "org.postgresql.Driver") \
  .mode("append")\
  .save()


Py4JJavaError: An error occurred while calling o90.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:25